In [17]:
import ssl

import pandas as pd
from sklearn.compose import make_column_selector as selector
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder

In [6]:
ssl._create_default_https_context = ssl._create_unverified_context

In [7]:
df = pd.read_csv(
    'https://raw.githubusercontent.com/INRIA/scikit-learn-mooc/main/'
    'datasets/adult-census.csv')
print(df.shape)
df.head()

(48842, 14)


,age,workclass,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,25,Private,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K


In [8]:
X = df
y = X.pop('class')

In [10]:
cat_col_selector = selector(dtype_include=object)
cat_cols = cat_col_selector(X)
X_cat = X[cat_cols]
X_cat.head()

,workclass,education,marital-status,occupation,relationship,race,sex,native-country
0,Private,11th,Never-married,Machine-op-inspct,Own-child,Black,Male,United-States
1,Private,HS-grad,Married-civ-spouse,Farming-fishing,Husband,White,Male,United-States
2,Local-gov,Assoc-acdm,Married-civ-spouse,Protective-serv,Husband,White,Male,United-States
3,Private,Some-college,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,United-States
4,?,Some-college,Never-married,?,Own-child,White,Female,United-States


In [14]:
mod = make_pipeline(
    OrdinalEncoder(
        handle_unknown='use_encoded_value', unknown_value=-1),
    LogisticRegression(max_iter=1000))
cv = cross_validate(mod, X_cat, y)

In [16]:
acc = cv['test_score'].mean()
sd = cv['test_score'].std()
print(f'Acc: {acc:.4f} ± {sd:.4f}')

Acc: 0.7555 ± 0.0015


In [20]:
mod2 = make_pipeline(
    OneHotEncoder(handle_unknown='ignore'),
    LogisticRegression(max_iter=1000))
cv2 = cross_validate(mod2, X_cat, y)
acc = cv2['test_score'].mean()
sd = cv2['test_score'].std()
print(f'Acc: {acc:.4f} ± {sd:.4f}')

Acc: 0.8329 ± 0.0024
